In [10]:
import dxpy
import dxdata
import subprocess
import ast
import numpy as np
import pandas as pd
import glob 
import os

In [11]:
dataset = 'record-GQ9v358JgBv5QBvgBXY9jb3F'
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

0

In [12]:
import glob
import os
import pandas as pd

path = os.getcwd()
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

from distutils.version import LooseVersion

def field_names_for_ids(field_id):
    field_names = ["eid"]
    for _id in field_id:
        select_field_names = list(data_dict_df[data_dict_df.name.str.match(r'^p{}(_i\d+)?(_a\d+)?$'.format(_id))].name.values)
        field_names += select_field_names
    field_names = sorted([field for field in field_names], key=lambda n: LooseVersion(n))
        
    field_names = [f"participant.{f}" for f in field_names]
    return ",".join(field_names)

In [13]:
field_ids = ['22006', #Genetic Ethnic Grouping
             '31', #Sex
             '21022',#Age at Recruitment
             '22009',#Genetic PCs
             '4080', #Systolic Blood Pressure
             '52', #Month of Birth
             '34', #Date of Birth
             '30710', #CReactive Protein
             '20116', #Smoking Status
             '21001',#BMI
             '30780',#LDL
             '20003',#Medication
             '137', #Medication_Groups
             '30700' #Creatinine
             
            ] 
field_names = field_names_for_ids(field_ids)
field_names

/tmp/ipykernel_97/2830623574.py:17: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  field_names = sorted([field for field in field_names], key=lambda n: LooseVersion(n))


'participant.eid,participant.p31,participant.p34,participant.p52,participant.p137_i0,participant.p137_i1,participant.p137_i2,participant.p137_i3,participant.p4080_i0_a0,participant.p4080_i0_a1,participant.p4080_i1_a0,participant.p4080_i1_a1,participant.p4080_i2_a0,participant.p4080_i2_a1,participant.p4080_i3_a0,participant.p4080_i3_a1,participant.p20003_i0,participant.p20003_i1,participant.p20003_i2,participant.p20003_i3,participant.p20116_i0,participant.p20116_i1,participant.p20116_i2,participant.p20116_i3,participant.p21001_i0,participant.p21001_i1,participant.p21001_i2,participant.p21001_i3,participant.p21022,participant.p22006,participant.p22009_a1,participant.p22009_a2,participant.p22009_a3,participant.p22009_a4,participant.p22009_a5,participant.p22009_a6,participant.p22009_a7,participant.p22009_a8,participant.p22009_a9,participant.p22009_a10,participant.p22009_a11,participant.p22009_a12,participant.p22009_a13,participant.p22009_a14,participant.p22009_a15,participant.p22009_a16,pa

In [15]:
field_names = 'participant.eid,participant.p31,participant.p34,participant.p52,participant.p4080_i0_a0,participant.p20116_i0,participant.p21001_i0,participant.p21022,participant.p22006,participant.p22009_a1,participant.p22009_a2,participant.p22009_a3,participant.p22009_a4,participant.p22009_a5,participant.p22009_a6,participant.p22009_a7,participant.p22009_a8,participant.p22009_a9,participant.p22009_a10,participant.p30700_i0,participant.p30710_i0,participant.p30780_i0,participant.p20003_i0,participant.p20003_i1,participant.p20003_i2,participant.p20003_i3,participant.p137_i0,participant.p137_i1,participant.p137_i2,participant.p137_i3'

In [16]:
cmd = ["dx", "extract_dataset", dataset, "--fields", field_names, "--delimiter", ",", "--output", "dispensed_dictionary.csv"]
subprocess.check_call(cmd)

0

In [17]:
dict_csv = "dispensed_dictionary.csv"
pdf = pd.read_csv(dict_csv)
import re
pdf = pdf.rename(columns=lambda x: re.sub('participant.','',x))

In [9]:
pdf.columns

Index(['eid', 'p31', 'p34', 'p52', 'p4080_i0_a0', 'p20116_i0', 'p21001_i0',
       'p21022', 'p22006', 'p22009_a1', 'p22009_a2', 'p22009_a3', 'p22009_a4',
       'p22009_a5', 'p22009_a6', 'p22009_a7', 'p22009_a8', 'p22009_a9',
       'p22009_a10', 'p30710_i0', 'p30780_i0', 'p20003_i0', 'p20003_i1',
       'p20003_i2', 'p20003_i3', 'p137_i0', 'p137_i1', 'p137_i2', 'p137_i3'],
      dtype='object')

In [18]:
pdf_qced = pdf.rename(columns=lambda x: re.sub('p22009_a','pc',x)) 
pdf_qced = pdf_qced.rename(columns=lambda x: re.sub('p20003_i','Med',x)) 
pdf_qced = pdf_qced.rename(columns=lambda x: re.sub('p137_i','RX_count',x)) 
pdf_qced = pdf_qced.rename(columns={'eid':'IID', 'p31': 'sex',
                        'p22006': 'ethnic_group',
                        'p21022': 'age',
                        'p34': 'year', 
                        'p52': 'month',
                        'p4080_i0_a0':'first_systolic_bp',
                        'p20116_i0':'smoking0',
                        'p21001_i0':'BMI',
                        'p30710_i0':'CReactive_protein',
                        'p30780_i0':'LDL_1',
                        'p30700_i0':'Creatinine_1'})

In [22]:
pdf_qced.columns

Index(['IID', 'sex', 'year', 'month', 'first_systolic_bp', 'smoking0', 'BMI',
       'age', 'ethnic_group', 'pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7',
       'pc8', 'pc9', 'pc10', 'Creatinine_1', 'CReactive_protein', 'LDL_1',
       'Med0', 'Med1', 'Med2', 'Med3', 'RX_count0', 'RX_count1', 'RX_count2',
       'RX_count3'],
      dtype='object')

In [23]:
filter_col = [col for col in pdf_qced if col.startswith('Med')]
pdf_qced[filter_col] = pdf_qced[filter_col].apply(lambda x: x.str.strip('[]'))
pdf_qced[filter_col] = pdf_qced[filter_col].apply(lambda x: x.str.strip())



In [24]:
pdf_qced

,IID,sex,year,month,first_systolic_bp,smoking0,BMI,age,ethnic_group,pc1,...,CReactive_protein,LDL_1,Med0,Med1,Med2,Med3,RX_count0,RX_count1,RX_count2,RX_count3
0,1000015,0,1954.0,3.0,136.0,0.0,36.9589,54.0,1.0,-14.7194,...,7.15,3.829,NaN,NaN,1140879802,NaN,0.0,NaN,1.0,NaN
1,1000064,1,1945.0,7.0,98.0,0.0,30.4201,62.0,1.0,-12.9614,...,1.55,2.630,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,1000096,1,1953.0,11.0,134.0,2.0,25.8891,54.0,1.0,-11.5948,...,18.22,2.399,"1141146138,1140860806","1140870390,1140868550,1140860806,1140861958",NaN,NaN,2.0,4.0,NaN,NaN
3,1000176,1,1960.0,10.0,151.0,1.0,25.2717,47.0,1.0,-12.8117,...,3.95,3.277,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,1000230,0,1963.0,3.0,NaN,0.0,24.4473,45.0,NaN,18.4585,...,0.46,4.416,1140869278,NaN,NaN,NaN,1.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502364,6022521,0,1966.0,12.0,101.0,0.0,23.2885,41.0,1.0,-14.0450,...,0.58,5.223,"1140870856,1140874866,1141169520,1140916682,11...",NaN,NaN,NaN,9.0,NaN,NaN,NaN
502365,6023002,0,1944.0,9.0,135.0,0.0,32.4265,64.0,1.0,-14.0832,...,2.24,3.116,"1140909674,1141188442",NaN,NaN,NaN,2.0,NaN,NaN,NaN
502366,6023375,0,1968.0,1.0,113.0,0.0,22.7915,42.0,1.0,-10.2602,...,0.53,2.393,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
502367,6023493,1,1947.0,9.0,138.0,0.0,21.8425,61.0,1.0,-11.0257,...,0.34,4.079,"1140868226,1140871310",NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [25]:
pdf_qced['day'] = 1
pdf_qced['birth_date'] = pd.to_datetime(pdf_qced[['year', 'month', 'day']])

pdf_qced.head(10)

,IID,sex,year,month,first_systolic_bp,smoking0,BMI,age,ethnic_group,pc1,...,Med0,Med1,Med2,Med3,RX_count0,RX_count1,RX_count2,RX_count3,day,birth_date
0,1000015,0,1954.0,3.0,136.0,0.0,36.9589,54.0,1.0,-14.7194,...,NaN,NaN,1140879802,NaN,0.0,NaN,1.0,NaN,1,1954-03-01
1,1000064,1,1945.0,7.0,98.0,0.0,30.4201,62.0,1.0,-12.9614,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1,1945-07-01
2,1000096,1,1953.0,11.0,134.0,2.0,25.8891,54.0,1.0,-11.5948,...,"1141146138,1140860806","1140870390,1140868550,1140860806,1140861958",NaN,NaN,2.0,4.0,NaN,NaN,1,1953-11-01
3,1000176,1,1960.0,10.0,151.0,1.0,25.2717,47.0,1.0,-12.8117,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1,1960-10-01
4,1000230,0,1963.0,3.0,NaN,0.0,24.4473,45.0,NaN,18.4585,...,1140869278,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1,1963-03-01
5,1000298,0,1962.0,11.0,98.0,1.0,23.6879,47.0,1.0,-11.8445,...,1140921600,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1,1962-11-01
6,1000324,1,1943.0,1.0,152.0,0.0,25.5347,65.0,1.0,-14.4342,...,"1140879802,1140868226,1140909786,1140923276,11...",NaN,NaN,NaN,5.0,NaN,NaN,NaN,1,1943-01-01
7,1000390,0,1952.0,3.0,139.0,1.0,22.6953,57.0,1.0,-13.9244,...,"1140868342,1140868226,1140867818,2038460150,11...",NaN,NaN,NaN,5.0,NaN,NaN,NaN,1,1952-03-01
8,1000401,0,1966.0,12.0,121.0,0.0,18.1661,41.0,NaN,NaN,...,"1141172620,1140864752,1140870164",NaN,NaN,NaN,3.0,NaN,NaN,NaN,1,1966-12-01
9,1000536,0,1955.0,9.0,173.0,2.0,31.8350,54.0,1.0,-11.2118,...,"1141167594,1140861998",NaN,NaN,NaN,2.0,NaN,NaN,NaN,1,1955-09-01


In [26]:
pdf_phenotype = pdf_qced.drop(['year', 'month', 'day'], axis=1)
pdf_phenotype.to_csv('example_survival_base_pheno.txt', sep='\t', na_rep='NA', index=False, quoting=3 )

In [27]:
%%bash
dx upload --destination ./ example_survival_base_pheno.txt

ID                          file-Gk725KQJz88Z4jVVq83Q2qQk
Class                       file
Project                     project-GQ9jJq0Jz88vkz66zppF73QP
Folder                      /
Name                        example_survival_base_pheno.txt
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Wed May 22 16:52:10 2024
Created by                  lzhang
 via the job                job-Gk71z58Jz88X09kXQpPYggk6
Last modified               Wed May 22 16:52:12 2024
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"
